# Notebook de Generación de Versos
En este fichero enfocado para la entrega final del proyecto, experimentaremos con diferentes modelos de generación de texto. Donde nuestro objetivo va a ser conseguir nuevos versos, llamados `aleya`, que aporten contenido al Corán.

El enfoque que tomaremos será realizar comparaciones entre modelos generativos especializados en el árabe y modelos generativos generalistas (que sirvan para todas los idiomas).

Por otra parte, usaremos una **Large Language Model** (`LLM`) para evaluar la cohesión, coherencia y otras demás métricas de los versos generados. Dado el caso de que no haya ningún modelo evaluador árabe disponible, implementaremos otras *LLMs* manualmente para realizar esta segunda tarea. 

(zerbait gehiago gehitu/aldatu dezakegu)

## Dependencias

In [1]:
# Dependencias
import torch
from decouple import config
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
login(config("HF_TOKEN"))

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


## Generación de Aleyas

Como ya hemos mencionado previamente, usaremos este apartado para evaluar si los modelos generalistas son capaces de hacer mejor language modeling que los modelos especializados en arabe.

Como nota diremos que aparte de las pruebas que se hacen aqui, en la sección de generación de topics + clustering ya se comprueba que modelos multilenguaje como ollama-gemma3:4b generan bastante bien texto en arabe. No solo eso sino que sus traducciones parecen ser bastante fieles despues de haberlas traducido manualmente. Sin embargo, aunque las traducciones fueran buenas, las comparaciones de cosine-similayity eran bastante inconcluyentes. Esto es debido a que probablemente la forma en la que creamos los embeddings no era la correcta (al final el arabe y el ingles son semánticamente muy diferentes).

Ahora usaremos el modelo Qwen3-4B para generar versos en arabe. Como nota, lo hemos extraido del benchmark **Arabic Broad Leaderboard (ABL) - The first comprehensive Leaderboard for Arabic LLMs** de huggingface. Ademas, usamos la version reducida del modelo (4B) ya que el original pesa bastante.

Este modelo es el Qwen3-4B-Instruct-2507, capaz de hablar multiples idiomas. En nuestro caso, como estamos contrastando modelos (preentenados), haremos una protocolo de evaluacion para contrastarlos.

También importaremos el modelo jais-family-2p7b-chat. La familia de modelos Jais es una serie completa de modelos lingüísticos extensos (LLM) bilingües inglés-árabe. Estos modelos están optimizados para destacar en árabe, con sólidas capacidades en inglés. Para simplicidad de la tarea, hemos querido seleccionar un modelo que sepa algo de ingles para que podamos hacer las intrucciones nosotros mismos.

En resumidas cuentas comparamos un modelo generalista multilingüe (Qwen) frente a un modelo bilingüe especializado en árabe (JAIS), manteniendo el prompt en inglés para controlar la variable de entrada.

In [2]:
# Creamos una configuración para generar texto para que la evaluación sea más precisa/justa
GEN_CFG = dict(
    max_new_tokens=160,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1,
    use_cache=True,
)
GEN_CFG_2 = dict(
    max_new_tokens=160,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1,
    use_cache=False,
)

# Cargamos el modelo
def load_chat_model(model_id, trust_remote_code=False):
    tok = AutoTokenizer.from_pretrained(model_id, trust_remote_code=trust_remote_code)
    mdl = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype="auto",
        trust_remote_code=trust_remote_code
    )
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    return tok, mdl

# Generamos texto
def generate_chat(model, tokenizer, user_text, system_text=None, gen_cfg=GEN_CFG):
    messages = []
    if system_text:
        messages.append({"role": "system", "content": system_text})
    messages.append({"role": "user", "content": user_text})

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    input_len = inputs["input_ids"].shape[-1]

    with torch.no_grad():
        out = model.generate(
            **inputs,
            pad_token_id=tokenizer.pad_token_id,
            **gen_cfg
        )

    return tokenizer.decode(out[0][input_len:], skip_special_tokens=True)

In [3]:
jais_id = "inceptionai/jais-family-2p7b-chat"
jais_tok, jais_model = load_chat_model(jais_id, trust_remote_code=True)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
qwen_id = "Qwen/Qwen3-4B-Instruct-2507"
qwen_tok, qwen_model = load_chat_model(qwen_id, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Vamos a hacer una breve prueba para comprobar que todo funciona correctamente.

In [ ]:
prompt = "Who are you?"

print("JAIS")
print(generate_chat(jais_model, jais_tok, prompt, None, gen_cfg=GEN_CFG_2))

print("\nQWEN")
print(generate_chat(qwen_model, qwen_tok, prompt, None, gen_cfg=GEN_CFG))

=== JAIS ===
I am an advanced artificial intelligence model designed to assist with a wide range of tasks and inquiries. My capabilities include providing information, answering questions, and offering recommendations based on available data and patterns. I strive to provide accurate, comprehensive, and thoughtful responses to all queries.

=== QWEN ===
I am Qwen, a large-scale language model independently developed by the Tongyi Lab under Alibaba Group. I can answer questions, create text such as stories, official documents, emails, scripts, perform logical reasoning, coding, and more. I can also express opinions and play games. I support 100 languages, including but not limited to Chinese, English, German, French, Spanish, etc. If you have any questions or need assistance, feel free to ask me anytime! 😊


## Evaluación del Texto Generado
Para la evaluación del texto generado en arabe usaremos un modelo generalista multilingüe (Falcon). Gracias a su formación bilingüe (árabe e inglés) y a su capacidad de razonamiento, Falcon nos da una evaluación más neutral de la corrección gramatical, la fluidez y la coherencia que otros modelos que pueden ser mas especializados en ciertos dominios.

Nuestra tarea de evaluación se dividirá en tres partes:

1. Una tarea explicativa de conceptos principalmente relacionados con la religión y la fe.
2. Una tarea para explicar ideas religiosas o versos Coránicos.
3. Una tarea de prompts creativos religiosos.

En estas tareas evaluaremos diferentes aspectos:
### Dimensiones lingüísticas
1. Corrección gramatical (morfología, concordancia)
2. Fluidez y naturalidad en árabe clásico / MSA
3. Ausencia de interferencia del inglés (calcos, estructuras raras)
4. Registro adecuado (formal / religioso)

### Dimensiones semántico-religiosas
1. Adecuación conceptual (no errores teológicos graves)
2. Uso apropiado del léxico religioso (رحمة، تقوى، عبادة, etc.)
3. Coherencia interna del texto

In [ ]:
concepts = ["mercy", "justice", "faith", "patience", "forgiveness"]

def concepts_explanation(concepts, model, tokenizer, system_text, gen_cfg):
    responses_concepts = []
    for i in range(len(concepts)):
        prompt = f'''Explain the concept {concepts[i]} in Islamic theology'''
        responses_concepts.append(generate_chat(model, tokenizer, prompt, system_text, gen_cfg))
    return responses_concepts

# Definimos como debe comportarse el modelo
SYSTEM_AR = "Answer in Modern Standard Arabic only. Use a formal religious register. Do not quote religious texts verbatim."

responses_concepts_jais = concepts_explanation(concepts, jais_model, jais_tok, SYSTEM_AR, GEN_CFG_2)
responses_concepts_qwen = concepts_explanation(concepts, qwen_model, qwen_tok, SYSTEM_AR, GEN_CFG)

0
1
2
3
4


In [ ]:
explanations = ["how mercy is understood in Islam during times of hardship and suffering",
                "the relationship between faith and ethical behavior in Islamic thought",
                "the importance of intention in Islamic religious practice",
                "the role of community and social responsibility in Islam",
                "the purpose of worship in Islam beyond ritual practice"
                ]

def show_explanations(explanations, model, tokenizer, system_text, gen_cfg):
    responses_explanations = []
    for i in range(len(explanations)):
        prompt = f'''Explain {explanations[i]}.'''
        responses_explanations.append(generate_chat(model, tokenizer, prompt, system_text, gen_cfg))
    return responses_explanations

# Definimos como debe comportarse el modelo
SYSTEM_AR = "Answer in Modern Standard Arabic only. Use a formal religious register. Do not quote religious texts verbatim."

responses_explanations_jais = show_explanations(explanations, jais_model, jais_tok, SYSTEM_AR, GEN_CFG_2)
responses_explanations_qwen = show_explanations(explanations, qwen_model, qwen_tok, SYSTEM_AR, GEN_CFG)

In [ ]:
creativity = ["short religious reflection (6–8 sentences) about mercy and compassion",
              "moral paragraph about patience and trust in God during difficult times",
              "short sermon-style paragraph about forgiveness and reconciliation",
              "religious reflection on how faith guides everyday life",
              "short religious text about justice and personal responsibility"]

def show_creativity(creativity, model, tokenizer, system_text, gen_cfg):
    responses_creativity = {}
    for i in range(len(creativity)):
        prompt = f'''Write a {creativity[i]}'''
        responses_creativity.append(generate_chat(model, tokenizer, prompt, system_text, gen_cfg))
    return responses_creativity

# Definimos como debe comportarse el modelo
SYSTEM_AR = "Answer in Modern Standard Arabic only. Use a formal religious register. Do not quote religious texts verbatim."

responses_creativity_jais = show_creativity(creativity, jais_model, jais_tok, SYSTEM_AR, GEN_CFG_2)
responses_creativity_qwen = show_creativity(creativity, qwen_model, qwen_tok, SYSTEM_AR, GEN_CFG)

In [7]:
#llama_id = "google/gemma-2-9b-it"
llama_id = "meta-llama/Meta-Llama-3-8B-Instruct"
llama_tokenizer, llama_model = load_chat_model(llama_id, trust_remote_code=True)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00002-of-00004.safetensors:  42%|####1     | 2.09G/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
def evaluate_concepts(concepts, model, tokenizer, gen_cfg):
    evaluation_concepts = []
    for i in range(len(concepts)):
        prompt = f'''You are an expert in Arabic language and Islamic studies.
                    Evaluate the following Arabic text.

                    Criteria:
                    1. Grammatical correctness
                    2. Fluency and naturalness
                    3. Appropriateness of religious register
                    4. Conceptual correctness

                    Give a score from 1 to 5 for each criterion and a short justification.

                    Text:
                    {concepts[i]}'''
        
        evaluation_concepts.append(generate_chat(model, tokenizer, prompt, None, gen_cfg))
    return evaluation_concepts

evaluation_concepts_jais = evaluate_concepts(responses_concepts_jais, llama_model, llama_tokenizer, GEN_CFG)
evaluation_concepts_qwen = evaluate_concepts(responses_concepts_qwen, llama_model, llama_tokenizer, GEN_CFG)

In [ ]:
def evaluate_explanations(explanations, model, tokenizer, gen_cfg):
    for i in range(len(explanations)):
        prompt = f'''You are an expert in Arabic language and Islamic studies.
                    Evaluate the following Arabic text.

                    Criteria:
                    1. Grammatical correctness
                    2. Fluency and naturalness
                    3. Appropriateness of religious register
                    4. Conceptual correctness

                    Give a score from 1 to 5 for each criterion and a short justification.

                    Text:
                    {explanations[i]}'''

evaluation_concepts_jais = evaluate_explanations(responses_explanations_jais, llama_model, llama_tokenizer, GEN_CFG)
evaluation_concepts_qwen = evaluate_explanations(responses_explanations_qwen, llama_model, llama_tokenizer, GEN_CFG)

In [ ]:
def evaluate_creativity(creativity, model, tokenizer, gen_cfg):
    for i in range(len(creativity)):

        prompt = f'''You are an expert in Arabic language and Islamic studies.
                    Evaluate the following Arabic text.

                    Criteria:
                    1. Grammatical correctness
                    2. Fluency and naturalness
                    3. Appropriateness of religious register
                    4. Conceptual correctness

                    Give a score from 1 to 5 for each criterion and a short justification.

                    Text:
                    {creativity[i]}'''

evaluation_concepts_jais = evaluate_creativity(responses_creativity_jais, llama_model, llama_tokenizer, GEN_CFG)
evaluation_concepts_qwen = evaluate_creativity(responses_creativity_qwen, llama_model, llama_tokenizer, GEN_CFG)